<a href="https://colab.research.google.com/github/imhyunho99/2023-1--Deaplearning_Framework/blob/main/(PR)2_Char_LSTM_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이번 코드는 Batch_First 모드로 구현했습니다.

import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
# Preprocessing string data
# alphabet(0-25), space(26), start(27), end(28) -> 29 chars (0-28)

string = "hello pytorch. how long can a rnn cell remember? show me your limit!"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)

char_len

35

In [ ]:
# String to onehot vector
# a -> [1 0 0 ... 0 0]

def string_to_onehot(string):
    start = np.zeros(shape=char_len ,dtype=int)
    end = np.zeros(shape=char_len ,dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len ,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero])
    output = np.vstack([start,end])
    return output

In [ ]:
# Onehot vector to word
# [1 0 0 ... 0 0] -> a 

def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [ ]:
# 하이퍼파라미터 설정
# 이번코드는 배치사이즈가 1보다 큰 경우에 대해 만들었습니다.
batch_size = 5

# seq_len는 바꿔도 학습은 되지만 테스트시 편의성을 위해 1로 설정했습니다.
seq_len = 1

# num_layers는 자유롭게 바꿀 수 있습니다.
num_layers = 3
input_size = char_len
hidden_size = 35 
lr = 0.01
num_epochs = 1000

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

print(one_hot.size())

torch.Size([70, 35])


In [ ]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
    def forward(self,input,hidden,cell):
        output,(hidden,cell) = self.lstm(input,(hidden,cell))
        return output,hidden,cell
    
    def init_hidden_cell(self):
        hidden = torch.zeros(num_layers, batch_size, hidden_size)
        cell = torch.zeros(num_layers, batch_size, hidden_size)
        return hidden,cell
    
rnn = RNN(input_size,hidden_size, num_layers)

In [ ]:
# Loss function & Optimizer

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [ ]:
j=0
input_data = one_hot[j:j+batch_size].view(batch_size,seq_len,input_size)
print(input_data.size())

hidden,cell = rnn.init_hidden_cell()
print(hidden.size(),cell.size())

output,hidden,cell = rnn(input_data,hidden,cell)
print(output.size(),hidden.size(),cell.size())



torch.Size([5, 1, 35])
torch.Size([3, 5, 35]) torch.Size([3, 5, 35])
torch.Size([5, 1, 35]) torch.Size([3, 5, 35]) torch.Size([3, 5, 35])


In [ ]:
unroll_len = one_hot.size()[0]//seq_len -1
for i in range(num_epochs):
    optimizer.zero_grad()
    hidden,cell = rnn.init_hidden_cell()
    
    loss = 0
    for j in range(unroll_len-batch_size+1):
        
        # batch size에 맞게 one-hot 벡터를 스택 합니다.
        # 예를 들어 batch size가 3이면 pytorch에서 pyt를 one-hot 벡터로 바꿔서 쌓고
        # 목표값으로 yto를 one-hot 벡터로 바꿔서 쌓는 과정입니다.
        input_data = torch.stack([one_hot[j+k:j+k+seq_len] for k in range(batch_size)],dim=0)
        label = torch.stack([one_hot[j+k+1:j+k+seq_len+1] for k in range(batch_size)],dim=0)
        
        input_data = input_data
        label = label
        
        output, hidden, cell = rnn(input_data,hidden,cell)
        loss += loss_func(output.view(1,-1),label.view(1,-1))
        
    loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(loss)

tensor(2.2964, grad_fn=<AddBackward0>)
tensor(1.7050, grad_fn=<AddBackward0>)
tensor(1.6302, grad_fn=<AddBackward0>)
tensor(1.4942, grad_fn=<AddBackward0>)
tensor(1.2585, grad_fn=<AddBackward0>)
tensor(0.9254, grad_fn=<AddBackward0>)
tensor(0.6147, grad_fn=<AddBackward0>)
tensor(0.2815, grad_fn=<AddBackward0>)
tensor(0.1354, grad_fn=<AddBackward0>)
tensor(0.0813, grad_fn=<AddBackward0>)
tensor(0.0567, grad_fn=<AddBackward0>)
tensor(0.0427, grad_fn=<AddBackward0>)
tensor(0.0347, grad_fn=<AddBackward0>)
tensor(0.0315, grad_fn=<AddBackward0>)
tensor(0.0274, grad_fn=<AddBackward0>)
tensor(0.0243, grad_fn=<AddBackward0>)
tensor(0.0219, grad_fn=<AddBackward0>)
tensor(0.0199, grad_fn=<AddBackward0>)
tensor(0.0184, grad_fn=<AddBackward0>)
tensor(0.0179, grad_fn=<AddBackward0>)
tensor(0.0167, grad_fn=<AddBackward0>)
tensor(0.0156, grad_fn=<AddBackward0>)
tensor(0.0149, grad_fn=<AddBackward0>)
tensor(0.0143, grad_fn=<AddBackward0>)
tensor(0.0139, grad_fn=<AddBackward0>)
tensor(0.0135, grad_fn=<A

In [ ]:
hidden,cell = rnn.init_hidden_cell()

for j in range(unroll_len-batch_size+1):
    input_data = torch.stack([one_hot[j+k:j+k+seq_len] for k in range(batch_size)],dim=0)
    label = torch.stack([one_hot[j+k+1:j+k+seq_len+1] for k in range(batch_size)],dim=0)

    input_data = input_data
    label = label
    
    output, hidden, cell = rnn(input_data,hidden,cell)
    for k in range(batch_size):
        print(onehot_to_word(output[k].data),end="")
        if j < unroll_len-batch_size:
            break


hello pytorch. how long can a rnn cell remember? show me your limit!1